## Notebook for spike sorting from .rhd data without using the Pipeline of mountainsort (mac channel has no pipeline)
Uses:
    - intan2kwik (https://github.com/zekearneodo/intan2kwik/blob/master/README.md)
    - mountainlab suite(https://github.com/flatironinstitute/mountainlab-js)
    - mountainsort https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md
    - mountainsort examples https://github.com/flatironinstitute/mountainsort_examples/blob/master/README.md

In [1]:
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
from importlib import reload

# mountainlab imports
from pipefinch.neural.convert import intan
from pipefinch.neural.sort.mountain import core as msc

import logging

# Setup the logger
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
ch.setFormatter(formatter)
logger.addHandler(ch)

logger.info('Logger set')

2018-10-17 18:03:18,950 - root - INFO - Logger set


In [2]:
os.environ['OMP_NUM_THREADS'] = '8'
os.environ['MKL_NUM_THREADS'] = '8'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

### Session parameters and raw files

In [2]:
exp_base = os.path.abspath('/data/experiment')

file_sys = {'exp_base': os.path.abspath('/data/experiment')}

sess_par = {'bird': 'o15g15',
           'sess': '20180906_rec_02',
           'rec': 0}

file_sys_defaults = {'exp_base': os.path.abspath('/data/experiment')}


def file_names(sess_par, file_sys=None):
    f_s = unroll_file_sys(file_sys_defaults, file_sys)    
    
    f_n = {}
    f_n['raw'] = os.path.join(f_s['raw'], bird)

In [3]:
exp_base

'/data/experiment'

In [5]:
raw_folder = os.path.join(exp_base, 'raw_data', sess_par['bird'], 'neural', sess_par['sess'])
logger.info('Raw folder {}'.format(raw_folder))
sess_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
logger.info('Found {} files'.format(len(sess_files)))

2018-10-17 22:17:44,830 - root - INFO - Raw folder /data/experiment/raw_data/o15g15/neural/20180906_rec_02
2018-10-17 22:17:44,834 - root - INFO - Found 483 files


In [6]:
rhd_files = glob.glob(os.path.join(raw_folder, '*.rhd'))
rhd_files.sort()
pd_f = pd.DataFrame(rhd_files, columns=['f_path'])

pd_f['stamp'] = pd_f['f_path'].apply(lambda x: int(os.path.split(x)[-1].split('.')[0].split('_')[-1]))

# Select some of the files (by span of the day, start-end in hhmmss)
sess_start = 0
sess_end = 235959
pd_session = pd_f.loc[(pd_f['stamp'] > sess_start) & (pd_f['stamp'] < sess_end), :]
pd_session.index.size

rhd_list = pd_session['f_path'].values.tolist()[:]

## Scripts for sorting

### prep the files with their nice formats, locations and names


In [7]:
rhd_list

['/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093115.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093215.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093315.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093415.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093515.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093615.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093715.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093815.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093915.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_094015.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_094115.rhd',
 '/data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_09421

In [9]:
ss_folder = os.path.join(exp_base, 'ss_data', sess_par['bird'], sess_par['sess'])

file_paths, out_folder = msc.make_paths(ss_folder)
os.makedirs(ss_folder, exist_ok=True)
#todo:
# meta and pars from intan files et al
s_f = 20000
ds_params = {'samplerate': s_f}

with open(file_paths['params'], 'w') as fp:
    json.dump(ds_params, fp)
    logger.info('Created session par files {}'.format(file_paths['params']))
    
# create the mda file from the rhd_list
intan_meta, samples_in = intan.intan_to_bin(rhd_list, file_paths['mda'], variant='mda')

2018-10-17 22:19:37,364 - root - INFO - Created session par files /data/experiment/ss_data/o15g15/20180906_rec_02/params.json
2018-10-17 22:19:37,396 - pipefinch.neural.convert - INFO - Creating the output file /data/experiment/ss_data/o15g15/20180906_rec_02/raw.mda
2018-10-17 22:19:37,397 - pipefinch.neural.convert - INFO - File 0/483
2018-10-17 22:19:37,397 - intan2kwik.intan.load_intan_jit - INFO - Reading rhd intan file /data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093115.rhd
notch filter: 100%|██████████| 23/23 [00:00<00:00, 24.28it/s]
2018-10-17 22:19:44,342 - pipefinch.neural.convert - INFO - File 1/483
2018-10-17 22:19:44,342 - intan2kwik.intan.load_intan_jit - INFO - Reading rhd intan file /data/experiment/raw_data/o15g15/neural/20180906_rec_02/rec_1_180906_093215.rhd
notch filter: 100%|██████████| 23/23 [00:00<00:00, 67.71it/s]
2018-10-17 22:19:50,965 - pipefinch.neural.convert - INFO - File 2/483
2018-10-17 22:19:50,966 - intan2kwik.intan.load_intan_ji

In [10]:
ss_folder

'/data/experiment/ss_data/o15g15/20180906_rec_02'

In [11]:
file_paths

{'mda': '/data/experiment/ss_data/o15g15/20180906_rec_02/raw.mda',
 'params': '/data/experiment/ss_data/o15g15/20180906_rec_02/params.json',
 'geom': '/data/experiment/ss_data/o15g15/20180906_rec_02/geom.csv',
 'filt': '/data/experiment/ss_data/o15g15/20180906_rec_02/sort_out/filt.mda.prv',
 'pre': '/data/experiment/ss_data/o15g15/20180906_rec_02/sort_out/pre.mda.prv',
 'firings': '/data/experiment/ss_data/o15g15/20180906_rec_02/sort_out/firings.mda',
 'firings_curated': '/data/experiment/ss_data/o15g15/20180906_rec_02/sort_out/firings_curated.mda',
 'cluster_metrics': '/data/experiment/ss_data/o15g15/20180906_rec_02/sort_out/cluster_metrics.json'}

## Sorting without pipelines ('run' each process, for mac/windows machines without the js installation)

In [12]:
msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1,detect_threshold=3, dispatch_method='run')

2018-10-17 23:11:26,972 - pipefinch.sort.mountain.comre - INFO - Bandpass filter


AttributeError: 'NoneType' object has no attribute 'addProcess'

## Sorting with pipelines ('add' each process, for machinew with js installation)

In [14]:
from mountainlab_pytools import mlproc as mlp
Pipeline=mlp.initPipeline()

JSProxyWidget(status='Not yet rendered')

In [15]:
with Pipeline:
    msc.sort_dataset(file_paths=file_paths, adjacency_radius=-1, detect_threshold=3, dispatch_method='add')

2018-10-18 09:14:03,595 - pipefinch.sort.mountain.comre - INFO - Bandpass filter
2018-10-18 09:14:03,598 - pipefinch.sort.mountain.comre - INFO - Whitening
2018-10-18 09:14:03,601 - pipefinch.sort.mountain.comre - INFO - Sorting
/home/ezequiel/repos/pipefinch/pipefinch/neural/sort/mountain/core.py:168: UserWarning: Will sort with no geometry input
  'freq_min': freq_min,
2018-10-18 09:14:03,608 - pipefinch.sort.mountain.comre - INFO - Getting cluster metrics
2018-10-18 09:14:03,614 - pipefinch.sort.mountain.comre - INFO - Automatically curating


Output()

Finished pipeline.


In [33]:
file_paths

{'mda': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/raw.mda',
 'params': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/params.json',
 'geom': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/geom.csv',
 'filt': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/filt.mda.prv',
 'pre': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/pre.mda.prv',
 'firings': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings.mda',
 'firings_curated': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/firings_curated.mda',
 'cluster_metrics': '/Users/zeke/experiment/ss_data/o3g3/awake_2017-09-08_2925/sort_out/cluster_metrics.json'}

In [10]:
cmd = 'env'
p = subprocess.Popen([cmd], stdout=subprocess.PIPE)
ans = p.communicate()
print(ans)

(b'MANPATH=/opt/local/share/man/:\nTERM_PROGRAM=Apple_Terminal\nSHELL=/bin/bash\nTERM=xterm-color\nTMPDIR=/var/folders/gk/7z94s6yj40n4ggn5dl62xhbc0000gn/T/\nCONDA_SHLVL=1\nApple_PubSub_Socket_Render=/private/tmp/com.apple.launchd.rxM5aew32h/Render\nCONDA_PROMPT_MODIFIER=(mountain) \nTERM_PROGRAM_VERSION=404\nTERM_SESSION_ID=E6D6F9C0-AA53-473E-B5D3-B4EE02E408C3\nUSER=zeke\nCONDA_EXE=/Users/zeke/anaconda3/bin/conda\nML_CONDA_PACKAGES_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab/packages\nSSH_AUTH_SOCK=/private/tmp/com.apple.launchd.ZGXuue9qT4/Listeners\nML_CONDA_DIR=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nPATH=/Users/zeke/anaconda3/envs/mountain/bin:/Users/zeke/google-cloud-sdk/bin:/opt/local/bin:/opt/local/sbin:/Library/Frameworks/Python.framework/Versions/3.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin\nCONDA_PREFIX=/Users/zeke/anaconda3/envs/mountain\nPWD=/Users/zeke/repos\nML_CONFIG_DIRECTORY=/Users/zeke/anaconda3/envs/mountain/etc/mountainlab\nLANG=en_US

In [16]:
show_args_dict = {'raw': file_paths['mda'], 
             'filt': file_paths['filt'],
             'pre': file_paths['pre'],
            'samplereate': ds_params['samplerate'],
            'firings': file_paths['firings_curated'],
            'cluster_metrics': file_paths['cluster_metrics']}

cmd_args = ['--{} {}'.format(k, v) for k, v in show_args_dict.items()]
cmd = 'qt-mountainview'

logger.info('View command is {}'.format([cmd, ' '.join(cmd_args)]))
p = subprocess.Popen([cmd, ' '.join(cmd_args)], stdout=subprocess.PIPE)


2018-09-19 17:26:29,145 - root - INFO - View command is ['qt-mountainview', '--raw /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/raw.mda --filt /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/filt.mda.prv --pre /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/pre.mda.prv --samplereate 20000 --firings /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/firings_curated.mda --cluster_metrics /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/cluster_metrics.json']


In [17]:
print(' '.join(['qt-mountainview'] + cmd_args))

qt-mountainview --raw /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/raw.mda --filt /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/filt.mda.prv --pre /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/pre.mda.prv --samplereate 20000 --firings /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/firings_curated.mda --cluster_metrics /Users/zeke/experiment/ss_data/o3g3/awake_20180908_2925/sort_out/cluster_metrics.json


# other viewing examples

ev-templates output/templates.mda.prv
ev-timeseries dataset/raw.mda.prv --firings output/firings.mda.prv --samplerate=30000
qt-mountainview --raw dataset/raw.mda.prv --filt output/filt.mda.prv --pre output/pre.mda.prv --samplerate 30000 --firings output/firings.mda.prv
